In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

from src.config import KTHParams
from src.utils import rolling_variance_proxy, summary_errors
from src.kth_ipp import generate_fine_series_from_coarse

df = pd.read_csv("data/histo_trafic.csv", sep=";", encoding="latin1")
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

df["tstamp_clean"] = df["tstamp"].str.replace(r"^[a-zA-Zéûîôàç]+\s+", "", regex=True)

month_map = {"janvier":"January","février":"February","mars":"March","avril":"April","mai":"May","juin":"June",
             "juillet":"July","août":"August","septembre":"September","octobre":"October","novembre":"November","décembre":"December"}
for fr, en in month_map.items():
    df["tstamp_clean"] = df["tstamp_clean"].str.replace(fr, en, regex=False)

df["tstamp"] = pd.to_datetime(df["tstamp_clean"], format="%d %B %Y")
df = df.drop(columns="tstamp_clean")
df = df.sort_values(["secteur", "tstamp"])
df["trafic_mbps"] = pd.to_numeric(df["trafic_mbps"], errors="coerce")
df = df.dropna(subset=["trafic_mbps"])

# we choose ONE sector for the sanity plot
sector_id = df.groupby("secteur")["trafic_mbps"].mean().sort_values().index[len(df["secteur"].unique())//2]
df_s = df[df["secteur"] == sector_id].sort_values("tstamp")
coarse = df_s["trafic_mbps"].to_numpy()


p = KTHParams(T=300.0, dt=5.0, tau=0.1, zeta=0.5, rolling_k=48, var_floor_ratio=0.05)
coarse_var = rolling_variance_proxy(
    coarse, k=p.rolling_k, var_floor_ratio=p.var_floor_ratio,
    tau=p.tau, zeta=p.zeta, T=p.T
)


fine, report, recon = generate_fine_series_from_coarse(coarse, coarse_var, p, seed=42)

print("Sector:", sector_id)
print(report)
print(summary_errors(coarse, recon))

plt.figure(figsize=(14,5))
plt.plot(coarse, label="Original coarse (Mbps)")
plt.plot(recon, label="Reconstructed coarse from fine (mean over dt)")
plt.title("Fig.6(a) sanity check: coarse vs reconstructed coarse")
plt.xlabel("Coarse slot index")
plt.ylabel("Mbps")
plt.legend()
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'imp'